# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f1e94f86c10>
├── 'a' --> tensor([[-0.5323,  0.4776, -0.7491],
│                   [ 1.3549,  0.0343,  1.3544]])
└── 'x' --> <FastTreeValue 0x7f1e94f86340>
    └── 'c' --> tensor([[-0.2117,  0.7837, -0.0941,  1.1636],
                        [-0.7641,  0.8690, -0.1287, -0.5410],
                        [-1.1782,  0.3352,  0.6391, -0.0390]])

In [4]:
t.a

tensor([[-0.5323,  0.4776, -0.7491],
        [ 1.3549,  0.0343,  1.3544]])

In [5]:
%timeit t.a

61.4 ns ± 0.0306 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f1e94f86c10>
├── 'a' --> tensor([[-0.4815, -0.2948,  0.1709],
│                   [-0.5001,  0.5058,  1.7564]])
└── 'x' --> <FastTreeValue 0x7f1e94f86340>
    └── 'c' --> tensor([[-0.2117,  0.7837, -0.0941,  1.1636],
                        [-0.7641,  0.8690, -0.1287, -0.5410],
                        [-1.1782,  0.3352,  0.6391, -0.0390]])

In [7]:
%timeit t.a = new_value

60.5 ns ± 0.0277 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-0.5323,  0.4776, -0.7491],
               [ 1.3549,  0.0343,  1.3544]]),
    x: Batch(
           c: tensor([[-0.2117,  0.7837, -0.0941,  1.1636],
                      [-0.7641,  0.8690, -0.1287, -0.5410],
                      [-1.1782,  0.3352,  0.6391, -0.0390]]),
       ),
)

In [10]:
b.a

tensor([[-0.5323,  0.4776, -0.7491],
        [ 1.3549,  0.0343,  1.3544]])

In [11]:
%timeit b.a

54.6 ns ± 0.0364 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 6.8740e-01,  4.5980e-01, -1.0638e-03],
               [-1.4098e+00, -2.2801e-01,  4.4546e-01]]),
    x: Batch(
           c: tensor([[-0.2117,  0.7837, -0.0941,  1.1636],
                      [-0.7641,  0.8690, -0.1287, -0.5410],
                      [-1.1782,  0.3352,  0.6391, -0.0390]]),
       ),
)

In [13]:
%timeit b.a = new_value

487 ns ± 0.168 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

835 ns ± 0.116 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.3 µs ± 18.1 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

137 µs ± 464 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

139 µs ± 591 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f1e94f80130>
├── 'a' --> tensor([[[-0.5323,  0.4776, -0.7491],
│                    [ 1.3549,  0.0343,  1.3544]],
│           
│                   [[-0.5323,  0.4776, -0.7491],
│                    [ 1.3549,  0.0343,  1.3544]],
│           
│                   [[-0.5323,  0.4776, -0.7491],
│                    [ 1.3549,  0.0343,  1.3544]],
│           
│                   [[-0.5323,  0.4776, -0.7491],
│                    [ 1.3549,  0.0343,  1.3544]],
│           
│                   [[-0.5323,  0.4776, -0.7491],
│                    [ 1.3549,  0.0343,  1.3544]],
│           
│                   [[-0.5323,  0.4776, -0.7491],
│                    [ 1.3549,  0.0343,  1.3544]],
│           
│                   [[-0.5323,  0.4776, -0.7491],
│                    [ 1.3549,  0.0343,  1.3544]],
│           
│                   [[-0.5323,  0.4776, -0.7491],
│                    [ 1.3549,  0.0343,  1.3544]]])
└── 'x' --> <FastTreeValue 0x7f1e94f80220>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32 µs ± 41.9 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f1defc7ed00>
├── 'a' --> tensor([[-0.5323,  0.4776, -0.7491],
│                   [ 1.3549,  0.0343,  1.3544],
│                   [-0.5323,  0.4776, -0.7491],
│                   [ 1.3549,  0.0343,  1.3544],
│                   [-0.5323,  0.4776, -0.7491],
│                   [ 1.3549,  0.0343,  1.3544],
│                   [-0.5323,  0.4776, -0.7491],
│                   [ 1.3549,  0.0343,  1.3544],
│                   [-0.5323,  0.4776, -0.7491],
│                   [ 1.3549,  0.0343,  1.3544],
│                   [-0.5323,  0.4776, -0.7491],
│                   [ 1.3549,  0.0343,  1.3544],
│                   [-0.5323,  0.4776, -0.7491],
│                   [ 1.3549,  0.0343,  1.3544],
│                   [-0.5323,  0.4776, -0.7491],
│                   [ 1.3549,  0.0343,  1.3544]])
└── 'x' --> <FastTreeValue 0x7f1e94f80070>
    └── 'c' --> tensor([[-0.2117,  0.7837, -0.0941,  1.1636],
                        [-0.7641,  0.8690, -0.1287, -0.5410],
                 

In [23]:
%timeit t_cat(trees)

30.4 µs ± 34.6 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

58.7 µs ± 160 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[-0.5323,  0.4776, -0.7491],
                [ 1.3549,  0.0343,  1.3544]],
       
               [[-0.5323,  0.4776, -0.7491],
                [ 1.3549,  0.0343,  1.3544]],
       
               [[-0.5323,  0.4776, -0.7491],
                [ 1.3549,  0.0343,  1.3544]],
       
               [[-0.5323,  0.4776, -0.7491],
                [ 1.3549,  0.0343,  1.3544]],
       
               [[-0.5323,  0.4776, -0.7491],
                [ 1.3549,  0.0343,  1.3544]],
       
               [[-0.5323,  0.4776, -0.7491],
                [ 1.3549,  0.0343,  1.3544]],
       
               [[-0.5323,  0.4776, -0.7491],
                [ 1.3549,  0.0343,  1.3544]],
       
               [[-0.5323,  0.4776, -0.7491],
                [ 1.3549,  0.0343,  1.3544]]]),
    x: Batch(
           c: tensor([[[-0.2117,  0.7837, -0.0941,  1.1636],
                       [-0.7641,  0.8690, -0.1287, -0.5410],
                       [-1.1782,  0.3352,  0.6391, -0.0390]],
         

In [26]:
%timeit Batch.stack(batches)

79.1 µs ± 172 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[-0.5323,  0.4776, -0.7491],
               [ 1.3549,  0.0343,  1.3544],
               [-0.5323,  0.4776, -0.7491],
               [ 1.3549,  0.0343,  1.3544],
               [-0.5323,  0.4776, -0.7491],
               [ 1.3549,  0.0343,  1.3544],
               [-0.5323,  0.4776, -0.7491],
               [ 1.3549,  0.0343,  1.3544],
               [-0.5323,  0.4776, -0.7491],
               [ 1.3549,  0.0343,  1.3544],
               [-0.5323,  0.4776, -0.7491],
               [ 1.3549,  0.0343,  1.3544],
               [-0.5323,  0.4776, -0.7491],
               [ 1.3549,  0.0343,  1.3544],
               [-0.5323,  0.4776, -0.7491],
               [ 1.3549,  0.0343,  1.3544]]),
    x: Batch(
           c: tensor([[-0.2117,  0.7837, -0.0941,  1.1636],
                      [-0.7641,  0.8690, -0.1287, -0.5410],
                      [-1.1782,  0.3352,  0.6391, -0.0390],
                      [-0.2117,  0.7837, -0.0941,  1.1636],
                      [-0.7641,  

In [28]:
%timeit Batch.cat(batches)

144 µs ± 230 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

317 µs ± 9.38 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
